Installations:

In [ ]:
!pip install ollama langchain sentence-transformers pypdf chromadb langchain_community
!curl -fsSL https://ollama.com/install.sh | sudo sh


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve > /dev/null 2>&1 &

In [ ]:
# models = [
#     "llama3",
#     "deepseek-r1",
#     "deepseek-r1:1.5b",
#     "qwen2.5",
#     "qwen2.5:1.5b",
#     "gemma3:1b",
#     "gemma3"
# ]

models = [
    # "deepseek-r1",
    "deepseek-r1:1.5b"
]

Running Services in Background:

In [ ]:
# !ollama pull llama3
# !ollama pull deepseek-r1
!ollama pull deepseek-r1:1.5b
# !ollama pull qwen2.5
# !ollama pull qwen2.5:1.5b
# !ollama pull gemma3:1b
# !ollama pull gemma3

Create Required Directories:

In [ ]:
import os
os.makedirs("data", exist_ok=True)
os.makedirs("result", exist_ok=True)
os.makedirs("docs", exist_ok=True)


Read data:

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Initialize Values:

In [9]:
messages = []
prompt_template_symbol = "Extract cryptocurrency symbols mentioned in the text (e.g., BTC, ETH, BCH). Return only the symbol names.\n\nText: "
prompt_template_timespan = "Extract the timespan from the text: Short term, Mid term, Long term, or Not mentioned. Return only the label.\n\nText: "
prompt_template_sentiment = "Extract the overall sentiment toward the symbol in the text. Return only: POSITIVE, NEGATIVE, or NEUTRAL.\n\nText: "

Read Json Messages

In [10]:
import json

def process_messages ():
  for i in range(1, 5):
    with open(f"/content/drive/MyDrive/telegram/{i}.json", 'r+') as f:
        chat = json.load(f)
        for item in chat["messages"]:
            if item["type"] != "message": continue
            text = "".join(entity["text"] for entity in item["text_entities"])
            if len(text) < 6: continue
            with open(f"./docs/{i}-{item['id']}.txt", "w+") as wr:
               wr.write(f"At: {item['date']} a message with following content published in channel {chat['name']}:\n{text}")
            messages.append({
              "group_id": chat["id"],
              "timestamp": item["date"],
              "id": item["id"],
              "text": text
            })


In [11]:
# import csv
# process_messages()
# fieldnames = messages[0].keys()

# with open('messages.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     writer.writerows(messages)

Setup Database Connection:

In [ ]:
import sqlite3
def create_connection():
    conn = None
    conn = sqlite3.connect("result/models-without-fine-tuning.db")
    print(f"Connected to SQLite version {sqlite3.version}")
    return conn

def create_table(conn):
    c = conn.cursor()
    statement = """CREATE TABLE IF NOT EXISTS results (
                id integer PRIMARY KEY,
                message text,"""
    for model in models:
      statement = statement + f"""{model}_symbols,
          {model}_symbols_sources,
          {model}_timespan,
          {model}_timespan_sources,
          {model}_sentiment,
          {model}_sentiment_sources,"""
    statement = statement + "timestamp text NOT NULL)"
    c.execute(statement.replace("-","_").replace(":","_").replace(".", "_"))

def insert_data(conn, data):
    statement = f"""INSERT INTO results(timestamp,message"""
    for model in models:
      statement = statement + f",{model}_symbols,{model}_timespan,{model}_sentiment,{model}_symbols_sources,{model}_timespan_sources,{model}_sentiment_sources"
    statement = statement + f""")
        VALUES(?,?"""
    for model in models:
            statement = statement + f",?,?,?,?,?,?"
    statement = statement + ") "
    # sql = f"""INSERT INTO results(timestamp,start_timestamp,end_timestamp,text,symbols,timespan,sentiment)
    #           VALUES(?,?,?,?,?,?,?) """
    cur = conn.cursor()
    cur.execute(statement.replace("-","_").replace(":","_").replace(".", "_"), data)
    conn.commit()
    return cur.lastrowid

Send Message To Model

In [ ]:
# import datetime
# from time import sleep
# if __name__ == "__main__":
#   process_messages()
#   conn = create_connection()
#   create_table(conn)
#   index = 0
#   for item in messages:
#     index = index+1
#     print(index)
#     # start_time = datetime.datetime.now().timestamp()
#     data = [item["timestamp"]]
#     for model in models:
#       symbol_prompt = prompt_template_symbol + item["text"]
#       symbol_respone = send_to_llm(symbol_prompt, model)
#       timespan_prompt = prompt_template_timespan + item["text"]
#       timespan_respone = send_to_llm(timespan_prompt, model)
#       sentiment_prompt = prompt_template_sentiment + item["text"]
#       sentiment_respone = send_to_llm(sentiment_prompt, model)
#       symbol = clean_ollama_response(symbol_respone)
#       timespan = clean_ollama_response(timespan_respone)
#       sentiment = clean_ollama_response(sentiment_respone)
#       end_time = datetime.datetime.now().timestamp()
#       data.append(symbol)
#       data.append(timespan)
#       data.append(sentiment)
#     insert_data(conn, tuple(data))
#     sleep(0.1)


In [ ]:
# from ollama import chat
# import re
# def clean_ollama_response(response_text):
#     # Remove <think>...</think> parts (including any text inside them)
#     cleaned_text = re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL)

#     # Remove extra whitespace (optional)
#     cleaned_text = cleaned_text.strip()

#     return cleaned_text

# def send_to_llm(message, model):
#     response: ChatResponse = chat(model=model, messages=[
#     {
#         'role': 'user',
#         'content': message,
#     },
#     ])
#     return response['message']['content']


Run Main Thread:

In [ ]:
# import datetime
# import re
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from typing import List, Dict, Any, Tuple
# from ollama import chat

# # Pre-compile regex patterns for better performance
# THINK_PATTERN = re.compile(r'<think>.*?</think>', flags=re.DOTALL)

# def clean_ollama_response(response_text: str) -> str:
#     """Optimized response cleaner with pre-compiled regex"""
#     return THINK_PATTERN.sub('', response_text).strip()

# def send_to_llm(message: str, model: str) -> str:
#     """Send message to LLM and return cleaned response"""
#     response = chat(model=model, messages=[{'role': 'user', 'content': message}])
#     return clean_ollama_response(response['message']['content'])

In [ ]:
# def process_model_prompts(model: str, text: str) -> Tuple[str, str, str]:
#     """Process all prompts for a single model in parallel"""
#     # Create executor outside the with block to reuse it
#     executor = ThreadPoolExecutor(max_workers=3)  # One for each prompt type

#     try:
#         # Submit all prompt tasks
#         future_symbol = executor.submit(
#             send_to_llm, prompt_template_symbol + text, model
#         )
#         future_timespan = executor.submit(
#             send_to_llm, prompt_template_timespan + text, model
#         )
#         future_sentiment = executor.submit(
#             send_to_llm, prompt_template_sentiment + text, model
#         )

#         # Wait for all futures to complete with error handling
#         symbol = future_symbol.result()
#         timespan = future_timespan.result()
#         sentiment = future_sentiment.result()

#         return symbol, timespan, sentiment

#     except Exception as e:
#         print(f"Error processing model {model}: {str(e)}")
#         # Return default values or re-raise as needed
#         return "ERROR", "ERROR", "ERROR"

#     finally:
#         executor.shutdown(wait=True)



# def main_parallel_processing():
#     """Main function with parallel processing"""
#     # Initialize your variables (these should be defined elsewhere)
#     process_messages()
#     samples = random.sample(messages, 1000)
#     print(len(samples))
#     # Initialize database connection
#     conn = create_connection()
#     create_table(conn)

#     # Process all messages - consider adjusting max_workers based on your system
#     with ThreadPoolExecutor(max_workers=4) as executor:
#         # Submit all message processing tasks
#         future_to_index = {
#             executor.submit(process_item, item, models): idx
#             for idx, item in enumerate(samples, 1)
#         }

#         # Process completed tasks as they finish
#         for future in as_completed(future_to_index):
#             idx = future_to_index[future]
#             try:
#                 data = future.result()
#                 insert_data(conn, tuple(data))
#                 print(f"Completed processing message {idx}")
#             except Exception as e:
#                 print(f"Error processing message {idx}: {str(e)}")

#     # Close database connection
#     conn.close()

# if __name__ == "__main__":
#     main_parallel_processing()

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
import random

def load_and_process_documents(directory_path):
    """
    Load and process documents from a directory
    """
    documents = []

    for file in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file)
        if file.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
        elif file.endswith('.txt'):
            loader = TextLoader(file_path)
            documents.extend(loader.load())

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    texts = text_splitter.split_documents(documents)

    return texts

def create_vector_store(texts, persist_directory='./chroma_db'):
    """
    Create and persist a vector store from processed texts
    """
    # Use HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create and persist Chroma vector store
    vectordb = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vectordb.persist()

    return vectordb

def initialize_ollama_llm(model_name="deepseek-r1:1.5b"):
    """
    Initialize Ollama with the specified model
    Available models can be listed with `ollama list`
    """
    llm = Ollama(
        model=model_name,
        temperature=0.7,
        top_p=0.9,
        repeat_penalty=1.1
    )
    return llm

def setup_qa_chain(llm, vectordb):
    """
    Set up the retrieval-augmented QA chain
    """
    # Custom prompt template
    prompt_template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Create retrieval QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(search_kwargs={'k': 3}),
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain

def initialize_rag_system(docs_directory, model_name="deepseek-r1:1.5b"):
    """
    Initialize the complete RAG system
    """
    # Process documents
    print("Processing documents...")
    texts = load_and_process_documents(docs_directory)

    # Create vector store
    print("Creating vector store...")
    vectordb = create_vector_store(texts)

    # Initialize LLM
    print(f"Initializing Ollama with {model_name}...")
    llm = initialize_ollama_llm(model_name)

    # Set up QA chain
    print("Setting up QA chain...")
    qa_chain = setup_qa_chain(llm, vectordb)

    print("RAG system ready!")
    return qa_chain

def query_rag_system(qa_chain, question):
    """
    Query the RAG system with a question
    """
    result = qa_chain({"query": question})

    print("\nAnswer:", result["result"])
    print("\nSources:")
    for doc in result["source_documents"]:
        print(f"- {doc.metadata['source']} (page {doc.metadata.get('page', 'N/A')})")

    return result

if __name__ == "__main__":
    # Initialize the RAG system
    docs_directory = "./docs"  # Replace with your documents directory

    process_messages()
    samples = random.sample(messages, 1000)
    print(len(samples))
    # Initialize database connection
    conn = create_connection()
    create_table(conn)
    qa_chain = initialize_rag_system(docs_directory, model_name="deepseek-r1:1.5b")



    # Example query
    for item in samples:
        data = [item["timestamp"], item["text"]]
        symbol_question = prompt_template_symbol + item["text"]
        timespan_question = prompt_template_timespan + item["text"]
        sentiment_question = prompt_template_sentiment + item["text"]
        symbol = query_rag_system(qa_chain, symbol_question)
        timespan = query_rag_system(qa_chain, timespan_question)
        sentiment = query_rag_system(qa_chain, sentiment_question)

        symbol_sources = "".join(doc.metadata['source'] for doc in symbol["source_documents"])
        timespan_sources = "".join(doc.metadata['source'] for doc in timespan["source_documents"])
        sentiment_sources = "".join(doc.metadata['source'] for doc in sentiment["source_documents"])

        data.extend([symbol["result"], timespan["result"],  sentiment["result"], symbol_sources, timespan_sources, sentiment_sources])
        insert_data(conn, tuple(data))

Streaming output truncated to the last 5000 lines.

However, the answer given was "Short term," which suggests that in this context, 25 minutes is considered a short term. But I'm not entirely sure because sometimes the time frame can be ambiguous without more specific information. It's possible that the user intended to refer to an hour or longer, but the way it's phrased here might make "short term" the correct answer.

I should double-check if there are any other timespans mentioned in the text. The first message is about 25 minutes left, which would fall under short term. So maybe the user is considering how long a trading strategy or plan can be. If someone is planning for several hours to prepare, they might refer to that as a long term.

But since the question is to extract just the label, and given that 25 minutes is less than an hour but more than five minutes, it's likely intended to categorize within the short term category, hence "Short term." I think that's what the user e